# Pytorch Implementation of EEGLearn - P. Bashivan

This notebook describes a short summary of Pytorch implementation of the models described in "Learning Representations from EEG with Deep Recurrent-Convolutional Neural Networks." Bashivan et al. at International conference on learning representations (2016).

The rest of the code is in the different python scripts of this repo.

All the codes have been inspired from the [original github](https://github.com/pbashivan/EEGLearn).

## Librairies Import

In [1]:
import numpy as np 
import scipy.io as sio
import torch
import os 

import torch.optim as optim
import torch.nn.functional as F

from torch.autograd import Variable
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader,random_split

from Utils import *
from Models import *

torch.manual_seed(1234)
np.random.seed(1234)

import warnings
warnings.simplefilter("ignore")

## Loading the original Images 
The images have directly been taken from original implementation, given that they remain the same nevermind the implementation (Pytorch, Tensorflow, Theano).

In [2]:
Mean_Images = sio.loadmat("Sample Data/images.mat")["img"] #corresponding to the images mean for all the seven windows
print(np.shape(Mean_Images)) 
Images = sio.loadmat("Sample Data/images_time.mat")["img"] #corresponding to the images mean for all the seven windows
print(np.shape(Images)) 
Label = (sio.loadmat("Sample Data/FeatureMat_timeWin")["features"][:,-1]-1).astype(int) #corresponding to the signal label (i.e. load levels).
print(np.shape(Label)) 
Patient_id = sio.loadmat("Sample Data/trials_subNums.mat")['subjectNum'][0] #corresponding to the patient id
print(np.shape(Patient_id))

(2670, 3, 32, 32)
(7, 2670, 3, 32, 32)
(2670,)
(2670,)


In [3]:
np.transpose(Images,(1,0,2,3,4)).shape

(2670, 7, 3, 32, 32)

In [4]:
Images_t = np.transpose(Images,(1,0,2,3,4))

## Loading patient dataset 
From the total data, we select the images corresponding patient. 

In [5]:
print("Choose among the patient : "+str(np.unique(Patient_id)))

Choose among the patient : [ 1  2  3  4  6  7  8  9 10 11 12 14 15]


In [6]:
choosen_patient = 8

## Introduction: BasicCNN
First Implementation of a CNN on the Mean Images from each patient

In [17]:
train_part = 0.8
test_part = 0.2

batch_size = 32

In [19]:
EEG = EEGImagesDataset(label=Label[Patient_id==choosen_patient], image=Mean_Images[Patient_id==choosen_patient])

lengths = [int(len(EEG)*train_part+1), int(len(EEG)*test_part)]
Train, Test = random_split(EEG, lengths)

Trainloader = DataLoader(Train,batch_size=batch_size)
Testloader = DataLoader(Test, batch_size=batch_size)

In [20]:
res = TrainTest_Model(BasicCNN, Trainloader, Testloader, n_epoch=50, learning_rate=0.001, print_epoch=-1, opti='Adam')

## Maxpool CNN
Build the Max-pooling model performing a maxpool over the 7 parallel convnets.

In [8]:
train_part = 0.8
test_part = 0.2

batch_size = 32

In [9]:
EEG = EEGImagesDataset(label=Label[Patient_id==choosen_patient], image=Images_t[Patient_id==choosen_patient])

lengths = [int(len(EEG)*train_part+1), int(len(EEG)*test_part)]
Train, Test = random_split(EEG, lengths)

Trainloader = DataLoader(Train,batch_size=batch_size)
Testloader = DataLoader(Test, batch_size=batch_size)

In [10]:
print('Begin Training for Patient '+str(choosen_patient))
res = TrainTest_Model(MaxCNN, Trainloader, Testloader, n_epoch=45, learning_rate=0.001, print_epoch=5, opti='Adam')

Begin Training for Patient 8
[5,  45]	loss: 0.790	Accuracy : 0.671		val-loss: 1.850	val-Accuracy : 0.526
[10,  45]	loss: 0.197	Accuracy : 0.935		val-loss: 0.661	val-Accuracy : 0.895
[15,  45]	loss: 0.411	Accuracy : 0.839		val-loss: 1.761	val-Accuracy : 0.737
[20,  45]	loss: 0.070	Accuracy : 0.981		val-loss: 0.697	val-Accuracy : 0.868
[25,  45]	loss: 0.003	Accuracy : 1.000		val-loss: 1.094	val-Accuracy : 0.842
[30,  45]	loss: 0.000	Accuracy : 1.000		val-loss: 1.327	val-Accuracy : 0.895
[35,  45]	loss: 0.000	Accuracy : 1.000		val-loss: 1.457	val-Accuracy : 0.921
[40,  45]	loss: 0.000	Accuracy : 1.000		val-loss: 1.536	val-Accuracy : 0.921
[45,  45]	loss: 0.000	Accuracy : 1.000		val-loss: 1.598	val-Accuracy : 0.921


## Temp CNN
FBuild the Conv1D model performing a convolution1D over the 7 parallel convnets.

In [9]:
print('Begin Training for Patient '+str(choosen_patient))
res = TrainTest_Model(TempCNN, Trainloader, Testloader, n_epoch=45, learning_rate=0.001, print_epoch=5, opti='Adam')

Begin Training for Patient 8
[5,  45]	loss: 0.723	Accuracy : 0.729		val-loss: 0.770	val-Accuracy : 0.658
[10,  45]	loss: 0.063	Accuracy : 0.968		val-loss: 0.314	val-Accuracy : 0.789
[15,  45]	loss: 0.046	Accuracy : 0.981		val-loss: 0.266	val-Accuracy : 0.895
[20,  45]	loss: 0.000	Accuracy : 1.000		val-loss: 0.403	val-Accuracy : 0.895
[25,  45]	loss: 0.000	Accuracy : 1.000		val-loss: 0.458	val-Accuracy : 0.895
[30,  45]	loss: 0.000	Accuracy : 1.000		val-loss: 0.477	val-Accuracy : 0.868
[35,  45]	loss: 0.000	Accuracy : 1.000		val-loss: 0.488	val-Accuracy : 0.868
[40,  45]	loss: 0.000	Accuracy : 1.000		val-loss: 0.501	val-Accuracy : 0.868
[45,  45]	loss: 0.000	Accuracy : 1.000		val-loss: 0.514	val-Accuracy : 0.868


## LSTM CNN
Build the LSTM model applying a RNN over the 7 parallel convnets outputs

In [8]:
EEG = EEGImagesDataset(label=Label[Patient_id==choosen_patient], image=Images_t[Patient_id==choosen_patient])

lengths = [int(len(EEG)*train_part+1), int(len(EEG)*test_part)]
Train, Test = random_split(EEG, lengths)

Trainloader = DataLoader(Train,batch_size=batch_size)
Testloader = DataLoader(Test, batch_size=batch_size)

In [9]:
print('Begin Training for Patient '+str(choosen_patient))
res = TrainTest_Model(LSTM, Trainloader, Testloader, n_epoch=45, learning_rate=0.0001, print_epoch=5, opti='Adam')

Begin Training for Patient 8
[5,  45]	loss: 1.367	Accuracy : 0.323		val-loss: 1.445	val-Accuracy : 0.158
[10,  45]	loss: 1.130	Accuracy : 0.490		val-loss: 1.197	val-Accuracy : 0.395
[15,  45]	loss: 0.721	Accuracy : 0.761		val-loss: 0.777	val-Accuracy : 0.737
[20,  45]	loss: 0.325	Accuracy : 0.916		val-loss: 0.548	val-Accuracy : 0.842
[25,  45]	loss: 0.157	Accuracy : 0.961		val-loss: 0.284	val-Accuracy : 0.868
[30,  45]	loss: 0.082	Accuracy : 0.987		val-loss: 0.152	val-Accuracy : 0.921
[35,  45]	loss: 0.031	Accuracy : 0.994		val-loss: 0.084	val-Accuracy : 0.974
[40,  45]	loss: 0.015	Accuracy : 1.000		val-loss: 0.080	val-Accuracy : 0.974
[45,  45]	loss: 0.010	Accuracy : 1.000		val-loss: 0.102	val-Accuracy : 0.974


## Mix CNN
Build the LSTM model applying a RNN and a CNN over the 7 parallel convnets outputs

In [8]:
EEG = EEGImagesDataset(label=Label[Patient_id==choosen_patient], image=Images_t[Patient_id==choosen_patient])

lengths = [int(len(EEG)*train_part+1), int(len(EEG)*test_part)]
Train, Test = random_split(EEG, lengths)

Trainloader = DataLoader(Train,batch_size=batch_size)
Testloader = DataLoader(Test, batch_size=batch_size)

In [10]:
print('Begin Training for Patient '+str(choosen_patient))
res = TrainTest_Model(Mix, Trainloader, Testloader, n_epoch=100, learning_rate=0.00001, print_epoch=5, opti='Adam')

Begin Training for Patient 8
[5, 100]	loss: 1.372	Accuracy : 0.323		val-loss: 1.417	val-Accuracy : 0.158
[10, 100]	loss: 1.368	Accuracy : 0.323		val-loss: 1.434	val-Accuracy : 0.158
[15, 100]	loss: 1.364	Accuracy : 0.323		val-loss: 1.441	val-Accuracy : 0.158
[20, 100]	loss: 1.348	Accuracy : 0.323		val-loss: 1.439	val-Accuracy : 0.158
[25, 100]	loss: 1.284	Accuracy : 0.323		val-loss: 1.426	val-Accuracy : 0.158
[30, 100]	loss: 1.160	Accuracy : 0.355		val-loss: 1.371	val-Accuracy : 0.237
[35, 100]	loss: 1.024	Accuracy : 0.619		val-loss: 1.284	val-Accuracy : 0.474
[40, 100]	loss: 0.897	Accuracy : 0.697		val-loss: 1.221	val-Accuracy : 0.553
[45, 100]	loss: 0.771	Accuracy : 0.781		val-loss: 1.149	val-Accuracy : 0.605
[50, 100]	loss: 0.637	Accuracy : 0.832		val-loss: 1.047	val-Accuracy : 0.737
[55, 100]	loss: 0.492	Accuracy : 0.871		val-loss: 0.908	val-Accuracy : 0.789
[60, 100]	loss: 0.360	Accuracy : 0.890		val-loss: 0.714	val-Accuracy : 0.789
[65, 100]	loss: 0.266	Accuracy : 0.897		val-loss